In [191]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
from ipywidgets import GridspecLayout,HTML,Layout,interactive_output,IntSlider 
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
from datetime import date
from datetime import datetime
plt.style.use('dark_background')

grid = GridspecLayout(6, 3)

title1 = widgets.HTML(
#     value="<h1>★★★★★  Parabolic Solar Dish Concentrator Calculator ★★★★★</h1>",
    value=f"<center><font size = 6><b><font color='black'>{'Parabolic Solar Dish Concentrator Calculator'}</b></center>",
)
title2 = widgets.HTML(
#     value="<h0>------------------------------------------- Desarrollado por Aguilar Hancco Junior Joel, estudiante de la carrera de Ingenieria Mecanica ---------------------------------------------------</h0>",
    value=f"<center><font size = 1><b><font color='black'>{'Desarrollado por Aguilar Hancco Junior Joel, estudiante de Ingenieria Mecanica'}</b></center>",
)
title3 = widgets.HTML(
    value=f"<center><font size = 3><b><font color='black'>{'Parametro de calculo '}</b></center>",
)
latitud = widgets.FloatSlider(
    value=-16.41833,
    min=-90,
    max=90,
    step=0.1,
    description='Latitud: ',
    disabled=False
)
hora = widgets.FloatSlider(
    value=12,
    min=0,
    max=24,
    step=24/72,
    description='Hora: ',
    disabled=False
)
altura = widgets.FloatSlider(
    value=2.405,
    min=0,
    max=2.5,
    step=0.1,
    description='M.s.n.m: ',
    disabled=False
)
latitud.style.handle_color = 'yellow'
hora.style.handle_color = 'yellow'
altura.style.handle_color = 'yellow'

grid[0,:] = title1
grid[1,:] = title2
grid[2,1] = title3
grid[3,0] = latitud
grid[3,1] = hora
grid[3,2] = altura

In [198]:
# -------------------------------------------------------------------------------------------------------------------------

# |------------------------------------------------------------|
# |  Funcion para mantener la relacion de los ejes iguales 3D  |
# |------------------------------------------------------------|
%matplotlib widget
def set_axes_equal(ax):

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

# -------------------------------------------------------------------------------------------------------------------------
    
# |-------------------------------------------------------------|
# |  Grafica del seguimiento del concentrador solar parabolico  |
# |-------------------------------------------------------------|

# lat = -16.41833 # Latitu del lugar
# lon = -71.48554 # Longitud del lugar

def pdsc(lat,hou):
        
    ndd      = 73 # Cantidad de angulos de posicion solar a calcular durante el presente dia
    height_z = 24
    radius   = 24
    elements = 30

    # Solar Time
    hoy = date.today()
    hor = datetime.now().hour
    mnt = datetime.now().minute
    sec = datetime.now().second
    doy = datetime.now().timetuple().tm_yday
    c0  = ((doy - 1) * 360) / 365
    c1  = 229.2 * (0.000075 + 0.001868 * np.cos(c0 * np.pi/180) - 0.032077 * np.sin(c0 * np.pi/180) - 0.014615 * np.cos(2 * c0 * np.pi/180) - 0.04089 * np.sin(2 * c0 * np.pi/180))
    c2  = (4 * 5 + c1) / 60
    
    hr  = np.linspace(0,24,ndd)
    stp = (24 - 0) / (ndd - 1)
    k   = int(np.fix((hou - 0) / stp + 1) - 1)
    
    Alt = np.zeros(len(hr))
    Azi = np.zeros(len(hr))
    Hor = []
    i   = -1

    for h in hr:
        i = i + 1
        w = 15 * (h + c2 - 12) # grados

    # Declination angle
        gam = np.arcsin(0.39795 * np.cos(0.98563 * (doy - 173)  * np.pi/180)) * 180/np.pi

    # Sun altitude angle
        Alt[i] = np.arcsin(np.sin(gam  * np.pi/180) * np.sin(lat  * np.pi/180) + np.cos(gam * np.pi/180) * np.cos(lat * np.pi/180) * np.cos(w * np.pi/180)) * 180/np.pi

    # Azimuth angle
        A   = np.arccos((np.sin(gam  * np.pi/180) * np.cos(lat  * np.pi/180) - np.cos(gam * np.pi/180) * np.sin(lat * np.pi/180) * np.cos(w * np.pi/180)) / np.cos(Alt[i] * np.pi/180)) * 180/np.pi
        if np.sin(w  * np.pi/180) > 0:
            Azi[i] = 360 - A
        else:
            Azi[i] = A
    
    h   = hor + mnt/60 + sec/3600 + c2
    w   = 15 * (h - 12) # grados

    # Declination angle
    gam = np.arcsin(0.39795 * np.cos(0.98563 * (doy - 173) * np.pi/180)) * 180/np.pi

    # Sun altitude angle
    alt = np.arcsin(np.sin(gam * np.pi/180) * np.sin(lat * np.pi/180) + np.cos(gam * np.pi/180) * np.cos(lat * np.pi/180) * np.cos(w * np.pi/180)) * 180/np.pi

    # Azimuth angle
    A   = np.arccos((np.sin(gam * np.pi/180) * np.cos(lat * np.pi/180) - np.cos(gam * np.pi/180) * np.sin(lat * np.pi/180) * np.cos(w * np.pi/180)) / np.cos(alt * np.pi/180)) * 180/np.pi
    if np.sin(w  * np.pi/180) > 0:
        azi = 360 - A
    else:
        azi = A

    fig4  = plt.figure(figsize=(10,10), dpi= 95)
    ax4   = Axes3D(fig4)

    u, v = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
    x = height_z * np.cos(u) * np.sin(v)
    y = height_z * np.sin(u) * np.sin(v)
    z = height_z * np.cos(v) + height_z 
    a = 5 * height_z  * np.cos(Azi * np.pi/180) * np.cos(Alt * np.pi/180)
    b = 5 * height_z  * np.sin(Azi * np.pi/180) * np.cos(Alt * np.pi/180)
    c = 5 * height_z  * np.sin(Alt * np.pi/180) + height_z
    ax4.plot_surface(x, y, z, cmap=plt.cm.YlGnBu_r, alpha = 0.25)
    ax4.plot(a, b, c, 'y', lw = 4)
    ax4.plot(0, 0, height_z, 'oc', markersize=10)  
        
    rota_azi = 360 - Azi[k] # rojo zy hora Z
    rota_alt = Alt[k] + 180 # azul xz anti Y
    center_x = height_z * np.cos(rota_alt * np.pi/180) * np.cos(rota_azi * np.pi/180)
    center_y = height_z * np.cos(rota_alt * np.pi/180) * np.sin(rota_azi * np.pi/180)
    center_z = height_z * np.sin(rota_alt * np.pi/180)
    center_X = 5 * height_z * np.cos(Alt[k] * np.pi/180) * np.cos((360 - Azi[k]) * np.pi/180)
    center_Y = 5 * height_z * np.cos(Alt[k] * np.pi/180) * np.sin((360 - Azi[k]) * np.pi/180)
    center_Z = 5 * height_z * np.sin(Alt[k] * np.pi/180)

    theta    = np.linspace(0, 2*np.pi, elements)
    radius   = np.linspace(0, radius, elements)
    z        = (radius ** 2) / (4 * height_z)
    theta_grid, z_grid = np.meshgrid(theta, z)
    x_grid             = [radius[i] * np.cos(theta_grid)[i][:] for i in range(0,elements)]
    y_grid             = [radius[i] * np.sin(theta_grid)[i][:] for i in range(0,elements)]
    x_grid   = np.array(x_grid)
    y_grid   = np.array(y_grid)

    rotation = -rota_alt - 90
    my = np.array([[np.cos(rotation * np.pi/180), 0, np.sin(rotation * np.pi/180)], [0, 1, 0], [-np.sin(rotation * np.pi/180), 0, np.cos(rotation * np.pi/180)]])

    for i in range(0,elements):
        for j in range(0,elements):
            xr = x_grid[i][j] 
            yr = y_grid[i][j] 
            zr = z_grid[i][j] 
            mc = np.array([xr, yr, zr])
            mr = np.dot(my, mc)
            x_grid[i][j] = mr[0]
            y_grid[i][j] = mr[1]
            z_grid[i][j] = mr[2]

    rotation = rota_azi
    mz = np.array([[np.cos(rotation * np.pi/180), -np.sin(rotation * np.pi/180), 0], [np.sin(rotation * np.pi/180), np.cos(rotation * np.pi/180), 0], [0, 0, 1]])

    for i in range(0,elements):
        for j in range(0,elements):
            xr = x_grid[i][j] 
            yr = y_grid[i][j] 
            zr = z_grid[i][j] 
            mc = np.array([xr, yr, zr])
            mr = np.dot(mz, mc)
            x_grid[i][j] = mr[0]
            y_grid[i][j] = mr[1]
            z_grid[i][j] = mr[2]

    x_grid = x_grid + center_x   
    y_grid = y_grid + center_y
    z_grid = z_grid + height_z + center_z

#     fig4  = plt.figure(figsize=(10,10), dpi= 95)
#     ax4   = Axes3D(fig4)

#     u, v = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
#     x = height_z * np.cos(u) * np.sin(v)
#     y = height_z * np.sin(u) * np.sin(v)
#     z = height_z * np.cos(v) + height_z 
#     ax4.plot_surface(x, y, z, cmap=plt.cm.YlGnBu_r, alpha = 0.25)

#     a = 5 * height_z  * np.cos(Azi * np.pi/180) * np.cos(Alt * np.pi/180)
#     b = 5 * height_z  * np.sin(Azi * np.pi/180) * np.cos(Alt * np.pi/180)
#     c = 5 * height_z  * np.sin(Alt * np.pi/180) + height_z
#     ax4.plot(a, b, c, 'y', lw = 4)
#     ax4.plot(0, 0, height_z, 'oc', markersize=10)
    
    ax4.plot([center_x, center_X], [center_y, center_Y], [height_z + center_z, height_z + center_Z], '--y', lw=2)
    ax4.plot(center_X, center_Y, height_z + center_Z, 'oy', markersize=20)
    ax4.plot_surface(x_grid, y_grid, z_grid, cmap = 'hot', alpha=0.8)
    
    ax4.set_title('Solar Tracker')
    ax4.set_xlabel('$x$')
    ax4.set_ylabel('$y$')
    ax4.set_zlabel('$z$') 
    ax4.view_init(elev=15, azim=130) 
    set_axes_equal(ax4)
    
widgets.interact(pdsc, lat = latitud, hou = hora);  
display(grid)

interactive(children=(FloatSlider(value=-72.6, description='Latitud: ', layout=Layout(grid_area='widget004'), …

GridspecLayout(children=(HTML(value="<center><font size = 6><b><font color='black'>Parabolic Solar Dish Concen…